In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import os

In [13]:
match_rep_df = pd.read_csv("checkpoints/match_rep_df.csv")

experiment_df = pd.read_csv("output/all-profile-metadata.csv")
experiment_df

,Vendor,Batch,Plate_Map_Name,Assay_Plate_Barcode,Modality,Images_per_well,Sites-SubSampled,Binning,Magnification,Number_of_channels,z_plane,BF_Zplanes,spinning-disc,aperture,dry-immersion,vs-brightfield,simultaneous-excitation,sites,Size_MB,Size_MB_std
0,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,4,NaN,1,10,6,1,NaN,no,0.45,dry,NaN,NaN,4,52.476334,0.000144
1,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,4,NaN,1,10,6,1,NaN,yes,0.45,dry,NaN,NaN,4,52.476514,0.000183
2,MolDev,Scope1_MolDev_10X_4siteZ,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crestz,Confocal,4,NaN,1,10,6,3,NaN,yes,0.45,dry,NaN,NaN,4,52.476631,0.000142
3,MolDev,Scope1_MolDev_20X_4site,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_20XPA_Crestz,Confocal,4,NaN,1,20,6,3,NaN,yes,0.75,dry,NaN,NaN,4,52.476629,0.000114
4,MolDev,Scope1_MolDev_20X_9site,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_9site_20XPA,Confocal,9,NaN,1,20,6,1,NaN,no,0.75,dry,NaN,NaN,9,52.476334,0.000153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch,JUMP-MOA_compound_platemap,BRO0117056_20x,Confocal,9,4.0,1,20,5,12,13,NaN,1.00,water,NaN,NaN,4,8.003585,0.000044
180,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch_12Z,JUMP-MOA_compound_platemap,BRO0117056_20xb,Confocal,9,4.0,1,20,5,12,15,NaN,1.00,water,NaN,NaN,4,8.003585,0.000044
181,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO0117059_20X,Confocal,9,4.0,1,20,6,12,15,NaN,1.00,water,NaN,NaN,4,7.954439,0.000583
182,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO01177034_20x,Confocal,9,4.0,1,20,6,12,15,NaN,1.00,water,NaN,NaN,4,8.003584,0.000014


In [116]:
# Find channel names from load_data_csvs

def find_channel_names(metadata_df, load_data_path):
    """
    Find channel names from the load_data_with_illum csv files

    Then, check the feature.csv.gz file to double-check that these column names 
    exist there too
    """
    plates = []
    df = metadata_df.copy()
    col = list()

    for root, dirs, files in os.walk(load_data_path):
        for file in files:
            if "load_data_with_illum.csv" in file:
                path = os.path.join(root, file)
                plate = os.path.split(path)[0].split("/")[-1]
                plates.append(plate)
                if plate not in metadata_df["Assay_Plate_Barcode"].values:
                    print(path, plate)
                load_data = pd.read_csv(path)
                columns = load_data.columns.tolist()
                col.append(columns)
                channels = sorted([i.replace("FileName_Orig", "") for i in columns if "FileName_Orig" in i])
                channel_string = ", ".join(channels)

                df.loc[df["Assay_Plate_Barcode"] == plate, "channel_names"] = channel_string
                if "FileName_OrigBrightField" in columns:
                    file_list = load_data["FileName_OrigBrightField"].values
                    z_pos = list(set([i[-10:-7] for i in file_list]))
                    print(plate, set(z_pos))
                    # z_pos[0] to make z_pos not an iterable
                    df.loc[df["Assay_Plate_Barcode"] == plate, "brightfield_z_plane"] = z_pos[0]
    # df["feature_channels_found"] = ""
    return df
    # return col

df = find_channel_names(experiment_df[:2], "../jump-scope/load_data_csv/")

def find_channels_in_features(metadata_df, profile_parent_dir="../jump-scope/profiles"):
    df = metadata_df.copy()
    for ind, row in metadata_df.iterrows():
        for root, dirs, files in os.walk(profile_parent_dir):
            load_path = os.path.join(profile_parent_dir, row["Batch"], row["Assay_Plate_Barcode"],  row["Assay_Plate_Barcode"]+".csv.gz")
            for file in files:
                if os.path.join(root, file) == load_path:
                    load_df = pd.read_csv(load_path)
                    channels = row["channel_names"].split(", ")
                    print(channels)
                    found_chan = []
                    for chan in channels:
                        bf_df = load_df.loc[:, load_df.columns.str.contains(chan)]
                        if len(bf_df) > 0:
                            print(chan)
                            found_chan.append(chan)
                    found_chan = ", ".join(found_chan)
                    print(found_chan, row["Assay_Plate_Barcode"])
                    df.loc[df["Assay_Plate_Barcode"] == row["Assay_Plate_Barcode"], "feature_channels_found"] = found_chan
    return df


# df = find_channel_names(experiment_df[:2], "../jump-scope/load_data_csv/")

find_channels_in_features(df)

# set(experiment_df.channel_names)
# df[~df["brightfield_z_plane"].isnull()][["Batch", "Assay_Plate_Barcode", "brightfield_z_plane"]]

# [set(i) for i in df]

# t.to_csv("test.csv")

../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_9site_20XPA_Crest/load_data_with_illum.csv Plate3_PCO_6ch_9site_20XPA_Crest
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_1site_10XPA_Crestz/load_data_with_illum.csv Plate3_PCO_6ch_1site_10XPA_Crestz
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_Adaptive_20XPA/load_data_with_illum.csv Plate3_PCO_6ch_Adaptive_20XPA
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_4site_10XPA_Crestz/load_data_with_illum.csv Plate3_PCO_6ch_4site_10XPA_Crestz
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_4site_20XPA_Crestz/load_data_with_illum.csv Plate3_PCO_6ch_4site_20XPA_Crestz
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_1site_10XPA_Crest/load_data_with_illum.csv Plate3_PCO_6ch_1site_10XPA_Crest
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate2-PCO-6ch-1site-10XPA/load_data_with_illum.csv Plate2-PCO-6ch-1site-10XPA
../j

,Vendor,Batch,Plate_Map_Name,Assay_Plate_Barcode,Modality,Images_per_well,Sites-SubSampled,Binning,Magnification,Number_of_channels,...,aperture,dry-immersion,vs-brightfield,simultaneous-excitation,sites,Size_MB,Size_MB_std,channel_names,brightfield_z_plane,feature_channels_found
0,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,4,NaN,1,10,6,...,0.45,dry,NaN,NaN,4,52.476334,0.000144,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA"
1,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,4,NaN,1,10,6,...,0.45,dry,NaN,NaN,4,52.476514,0.000183,"Actin, DNA, ER, Golgi, Mito, RNA",NaN,"Actin, DNA, ER, Golgi, Mito, RNA"


In [115]:
t = "one, two, three"
t.split(", ")

['one', 'two', 'three']

In [74]:
# Wait, so some load_data.csv files load brightfield, but the metadata says they don't have BF? So which one is right? Let's check the profile features

bf_none = df[(df["BF_Zplanes"] == "none") & (df["channel_names"].str.contains("BrightField"))]
profile_parent_dir = "../jump-scope/profiles"

for ind, row in bf_none.iterrows():
    for root, dirs, files in os.walk(profile_parent_dir):
        load_path = os.path.join(profile_parent_dir, row["Batch"], row["Assay_Plate_Barcode"],  row["Assay_Plate_Barcode"]+"_normalized_feature_select_negcon_batch.csv.gz")
        for file in files:
            if os.path.join(root, file) == load_path:
                load_df = pd.read_csv(load_path)
                bf_df = load_df.loc[:, load_df.columns.str.contains("BrightField")]
                input_columns = bf_df.columns
                if len(input_columns) > 0:
                    print(f"{file} contains brightfield features, eg: {input_columns[0]}")

BRO0117014_10x_normalized_feature_select_negcon_batch.csv.gz contains brightfield features, eg: Cells_Correlation_Correlation_BrightField_DNA
BRO0117059_20X_normalized_feature_select_negcon_batch.csv.gz contains brightfield features, eg: Cells_Correlation_Manders_BrightField_DNA
BRO01177034_20x_normalized_feature_select_negcon_batch.csv.gz contains brightfield features, eg: Cells_Correlation_Overlap_BrightField_DNA
BRO0117059_40x_normalized_feature_select_negcon_batch.csv.gz contains brightfield features, eg: Cells_Correlation_Correlation_BrightField_DNA


Ok so, metadata is wrong

In [69]:
df.loc[:, df.columns.str.contains("Batch")]

,Batch
0,Scope1_MolDev_10X
1,Scope1_MolDev_10X
2,Scope1_MolDev_10X_4siteZ
3,Scope1_MolDev_20X_4site
4,Scope1_MolDev_20X_9site
...,...
179,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch
180,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch_12Z
181,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...
182,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...
